In [ ]:
import numpy as np
from scipy.interpolate import interp1d

### Function definition

In [ ]:
def extract_waveform_features_1(waveform, interpolation_factor):
    """
    Extract features from a waveform:
    1. AP Peak Width (Full-Width Half Maximum of the peak)
    2. Trough to Peak Duration
    
    Parameters:
    waveform (numpy array): The waveform signal
    interpolation_factor (int): Factor by which to interpolate the waveform
    
    Returns:
    features (numpy array): Array containing the AP Peak Width and Trough to Peak Duration
    """
    features = np.zeros(2)
    
    # Original and interpolated time vectors
    original_time = np.arange(len(waveform))
    interpolated_time = np.linspace(0, len(waveform) - 1, len(waveform) * interpolation_factor)
    
    # Interpolate waveform using cubic spline
    interp_func = interp1d(original_time, waveform, kind='cubic')
    interpolated_waveform = interp_func(interpolated_time)
    
    # 1. AP Peak Width (Full-Width Half Maximum)
    peak_index = np.argmax(interpolated_waveform)
    half_max = interpolated_waveform[peak_index] / 2
    
    # Find left and right indices around peak that reach half maximum
    left_indices = np.where(interpolated_waveform[:peak_index] >= half_max)[0]
    right_indices = np.where(interpolated_waveform[peak_index:] >= half_max)[0] + peak_index
    
    if left_indices.size > 0 and right_indices.size > 0:
        left_index = left_indices[0]
        right_index = right_indices[-1]
        features[0] = (right_index - left_index) / interpolation_factor
    else:
        features[0] = np.nan  # Assign NaN if no full-width at half maximum found
    
    # 2. Trough to Peak Duration
    trough_index = np.argmin(interpolated_waveform)
    if peak_index > trough_index:
        features[1] = (peak_index - trough_index) / interpolation_factor
    else:
        features[1] = np.nan  # Assign NaN if peak does not follow trough
    
    return features
